In [1]:
import os
import argparse
import logging

import torch
from torch.optim.lr_scheduler import StepLR
import torchtext

import seq2seq
from seq2seq.trainer import SupervisedTrainer
from seq2seq.models import EncoderRNN, DecoderRNN, Seq2seq
from seq2seq.loss import Perplexity
from seq2seq.optim import Optimizer
from seq2seq.dataset import SourceField, TargetField
from seq2seq.evaluator import Predictor
from seq2seq.util.checkpoint import Checkpoint

In [2]:
try:
    raw_input          # Python 2
except NameError:
    raw_input = input  # Python 3

In [3]:
src = SourceField()
tgt = TargetField()

In [4]:
type(src)

seq2seq.dataset.fields.SourceField

In [5]:
class parse_args:
    def __init__ (self):
        self.train_path='datasets/toy_reverse/train/data.txt'
        self.dev_path='datasets/toy_reverse/dev/data.txt'
        self.expt_dir='experiments/'
        self.log_level='info'
        self.load_checkpoint=None
        self.resume=False
        

In [6]:
opt=parse_args()

In [7]:
opt.resume

False

In [9]:
src = SourceField()
tgt = TargetField()
max_len = 50
def len_filter(example):
    return len(example.src) <= max_len and len(example.tgt) <= max_len
train = torchtext.data.TabularDataset(
    path=opt.train_path, format='tsv',
    fields=[('src', src), ('tgt', tgt)],
    filter_pred=len_filter
)
dev = torchtext.data.TabularDataset(
    path=opt.dev_path, format='tsv',
    fields=[('src', src), ('tgt', tgt)],
    filter_pred=len_filter
)
src.build_vocab(train, max_size=50000)
tgt.build_vocab(train, max_size=50000)
input_vocab = src.vocab
output_vocab = tgt.vocab

In [12]:
weight = torch.ones(len(tgt.vocab))
pad = tgt.vocab.stoi[tgt.pad_token]
loss = Perplexity(weight, pad)
if torch.cuda.is_available():
    loss.cuda()

In [13]:
seq2seq = None
optimizer = None

In [14]:
hidden_size=128
bidirectional = True
encoder = EncoderRNN(len(src.vocab), max_len, hidden_size,
                     bidirectional=bidirectional, variable_lengths=True)
decoder = DecoderRNN(len(tgt.vocab), max_len, hidden_size * 2 if bidirectional else hidden_size,
                     dropout_p=0.2, use_attention=True, bidirectional=bidirectional,
                     eos_id=tgt.eos_id, sos_id=tgt.sos_id)

In [15]:
seq2seq = Seq2seq(encoder, decoder)
if torch.cuda.is_available():
    seq2seq.cuda()

In [16]:
for param in seq2seq.parameters():
    param.data.uniform_(-0.08, 0.08)

In [17]:
# train
t = SupervisedTrainer(loss=loss, batch_size=32,
                      checkpoint_every=50,
                      print_every=1, expt_dir=opt.expt_dir)

seq2seq = t.train(seq2seq, train,
                  num_epochs=6, dev_data=dev,
                  optimizer=optimizer,
                  teacher_forcing_ratio=0.5,
                  resume=opt.resume)

2018-02-22 08:57:41,752 seq2seq.trainer.supervised_trainer INFO     Optimizer: <torch.optim.adam.Adam object at 0x7fedd0d70080>, Scheduler: None
/home/ubuntu/pytorch-seq2seq/seq2seq/models/attention.py:62: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attn = F.softmax(attn.view(-1, input_size)).view(batch_size, -1, input_size)
/home/ubuntu/pytorch-seq2seq/seq2seq/models/DecoderRNN.py:105: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  predicted_softmax = function(self.out(output.view(-1, self.hidden_size))).view(batch_size, output_size, -1)
2018-02-22 08:57:56,947 seq2seq.trainer.supervised_trainer INFO     Progress: 0%, Train Perplexity: 27.1630
2018-02-22 08:57:56,959 seq2seq.trainer.supervised_trainer INFO     Progress: 0%, Train Perplexity: 12.6644
2018-02-22 08:57:56,981 seq2seq.trainer.supervised_trainer INFO     Progress: 0%

2018-02-22 08:57:58,352 seq2seq.trainer.supervised_trainer INFO     Progress: 3%, Train Perplexity: 3.0245
2018-02-22 08:57:58,366 seq2seq.trainer.supervised_trainer INFO     Progress: 3%, Train Perplexity: 4.4555
2018-02-22 08:57:58,378 seq2seq.trainer.supervised_trainer INFO     Progress: 3%, Train Perplexity: 3.9171
2018-02-22 08:57:58,405 seq2seq.trainer.supervised_trainer INFO     Progress: 3%, Train Perplexity: 4.1676
2018-02-22 08:57:58,416 seq2seq.trainer.supervised_trainer INFO     Progress: 3%, Train Perplexity: 3.3472
2018-02-22 08:57:58,426 seq2seq.trainer.supervised_trainer INFO     Progress: 3%, Train Perplexity: 2.0668
2018-02-22 08:57:58,435 seq2seq.trainer.supervised_trainer INFO     Progress: 3%, Train Perplexity: 1.7590
2018-02-22 08:57:58,449 seq2seq.trainer.supervised_trainer INFO     Progress: 3%, Train Perplexity: 6.0613
2018-02-22 08:57:58,469 seq2seq.trainer.supervised_trainer INFO     Progress: 4%, Train Perplexity: 2.8589
2018-02-22 08:57:58,480 seq2seq.train

2018-02-22 08:57:59,779 seq2seq.trainer.supervised_trainer INFO     Progress: 7%, Train Perplexity: 1.5257
2018-02-22 08:57:59,793 seq2seq.trainer.supervised_trainer INFO     Progress: 7%, Train Perplexity: 1.7423
2018-02-22 08:57:59,804 seq2seq.trainer.supervised_trainer INFO     Progress: 7%, Train Perplexity: 1.9491
2018-02-22 08:57:59,831 seq2seq.trainer.supervised_trainer INFO     Progress: 7%, Train Perplexity: 2.8225
2018-02-22 08:57:59,841 seq2seq.trainer.supervised_trainer INFO     Progress: 7%, Train Perplexity: 1.1865
2018-02-22 08:57:59,853 seq2seq.trainer.supervised_trainer INFO     Progress: 7%, Train Perplexity: 1.6284
2018-02-22 08:57:59,910 seq2seq.trainer.supervised_trainer INFO     Progress: 8%, Train Perplexity: 1.6677
2018-02-22 08:57:59,920 seq2seq.trainer.supervised_trainer INFO     Progress: 8%, Train Perplexity: 1.2096
2018-02-22 08:57:59,931 seq2seq.trainer.supervised_trainer INFO     Progress: 8%, Train Perplexity: 1.4990
2018-02-22 08:57:59,945 seq2seq.train

2018-02-22 08:58:01,134 seq2seq.trainer.supervised_trainer INFO     Progress: 11%, Train Perplexity: 1.0087
2018-02-22 08:58:01,146 seq2seq.trainer.supervised_trainer INFO     Progress: 11%, Train Perplexity: 1.0015
2018-02-22 08:58:01,156 seq2seq.trainer.supervised_trainer INFO     Progress: 11%, Train Perplexity: 1.0338
2018-02-22 08:58:01,165 seq2seq.trainer.supervised_trainer INFO     Progress: 11%, Train Perplexity: 1.0012
2018-02-22 08:58:01,179 seq2seq.trainer.supervised_trainer INFO     Progress: 12%, Train Perplexity: 1.0134
2018-02-22 08:58:01,190 seq2seq.trainer.supervised_trainer INFO     Progress: 12%, Train Perplexity: 1.0171
2018-02-22 08:58:01,221 seq2seq.trainer.supervised_trainer INFO     Progress: 12%, Train Perplexity: 1.0581
2018-02-22 08:58:01,233 seq2seq.trainer.supervised_trainer INFO     Progress: 12%, Train Perplexity: 1.0291
2018-02-22 08:58:01,244 seq2seq.trainer.supervised_trainer INFO     Progress: 12%, Train Perplexity: 1.0078
2018-02-22 08:58:01,258 seq2

2018-02-22 08:58:02,500 seq2seq.trainer.supervised_trainer INFO     Progress: 15%, Train Perplexity: 1.0017
2018-02-22 08:58:02,513 seq2seq.trainer.supervised_trainer INFO     Progress: 15%, Train Perplexity: 1.0150
2018-02-22 08:58:02,535 seq2seq.trainer.supervised_trainer INFO     Progress: 15%, Train Perplexity: 1.0032
2018-02-22 08:58:02,598 seq2seq.trainer.supervised_trainer INFO     Progress: 16%, Train Perplexity: 9.4719
2018-02-22 08:58:02,623 seq2seq.trainer.supervised_trainer INFO     Progress: 16%, Train Perplexity: 1.0042
2018-02-22 08:58:02,634 seq2seq.trainer.supervised_trainer INFO     Progress: 16%, Train Perplexity: 1.0005
2018-02-22 08:58:02,646 seq2seq.trainer.supervised_trainer INFO     Progress: 16%, Train Perplexity: 1.0020
2018-02-22 08:58:02,674 seq2seq.trainer.supervised_trainer INFO     Progress: 16%, Train Perplexity: 1.0311
2018-02-22 08:58:02,688 seq2seq.trainer.supervised_trainer INFO     Progress: 16%, Train Perplexity: 11.6810
2018-02-22 08:58:02,722 seq

...

2018-02-22 08:58:26,672 seq2seq.trainer.supervised_trainer INFO     Progress: 80%, Train Perplexity: 1.0000
2018-02-22 08:58:26,681 seq2seq.trainer.supervised_trainer INFO     Progress: 80%, Train Perplexity: 1.0000
2018-02-22 08:58:26,690 seq2seq.trainer.supervised_trainer INFO     Progress: 80%, Train Perplexity: 1.0000
2018-02-22 08:58:26,703 seq2seq.trainer.supervised_trainer INFO     Progress: 80%, Train Perplexity: 1.0002
2018-02-22 08:58:26,725 seq2seq.trainer.supervised_trainer INFO     Progress: 80%, Train Perplexity: 1.0027
2018-02-22 08:58:26,736 seq2seq.trainer.supervised_trainer INFO     Progress: 80%, Train Perplexity: 1.0000
2018-02-22 08:58:26,747 seq2seq.trainer.supervised_trainer INFO     Progress: 80%, Train Perplexity: 1.0027
2018-02-22 08:58:26,761 seq2seq.trainer.supervised_trainer INFO     Progress: 80%, Train Perplexity: 1.0003
2018-02-22 08:58:26,772 seq2seq.trainer.supervised_trainer INFO     Progress: 80%, Train Perplexity: 1.0007
2018-02-22 08:58:26,785 seq2

2018-02-22 08:58:28,412 seq2seq.trainer.supervised_trainer INFO     Progress: 84%, Train Perplexity: 1.0000
2018-02-22 08:58:28,423 seq2seq.trainer.supervised_trainer INFO     Progress: 84%, Train Perplexity: 1.0000
2018-02-22 08:58:28,456 seq2seq.trainer.supervised_trainer INFO     Progress: 84%, Train Perplexity: 1.5713
2018-02-22 08:58:28,467 seq2seq.trainer.supervised_trainer INFO     Progress: 84%, Train Perplexity: 1.0000
2018-02-22 08:58:28,477 seq2seq.trainer.supervised_trainer INFO     Progress: 84%, Train Perplexity: 1.0000
2018-02-22 08:58:28,488 seq2seq.trainer.supervised_trainer INFO     Progress: 84%, Train Perplexity: 1.1012
2018-02-22 08:58:28,519 seq2seq.trainer.supervised_trainer INFO     Progress: 84%, Train Perplexity: 1.0204
2018-02-22 08:58:28,533 seq2seq.trainer.supervised_trainer INFO     Progress: 84%, Train Perplexity: 1.7759
2018-02-22 08:58:28,547 seq2seq.trainer.supervised_trainer INFO     Progress: 84%, Train Perplexity: 1.0981
2018-02-22 08:58:28,558 seq2

2018-02-22 08:58:29,744 seq2seq.trainer.supervised_trainer INFO     Progress: 88%, Train Perplexity: 1.7953
2018-02-22 08:58:29,758 seq2seq.trainer.supervised_trainer INFO     Progress: 88%, Train Perplexity: 1.0000
2018-02-22 08:58:29,781 seq2seq.trainer.supervised_trainer INFO     Progress: 88%, Train Perplexity: 1.0000
2018-02-22 08:58:29,791 seq2seq.trainer.supervised_trainer INFO     Progress: 88%, Train Perplexity: 1.0002
2018-02-22 08:58:29,802 seq2seq.trainer.supervised_trainer INFO     Progress: 88%, Train Perplexity: 1.0041
2018-02-22 08:58:29,824 seq2seq.trainer.supervised_trainer INFO     Progress: 88%, Train Perplexity: 1.0000
2018-02-22 08:58:29,837 seq2seq.trainer.supervised_trainer INFO     Progress: 88%, Train Perplexity: 4.5064
2018-02-22 08:58:29,865 seq2seq.trainer.supervised_trainer INFO     Progress: 88%, Train Perplexity: 1.1948
2018-02-22 08:58:29,877 seq2seq.trainer.supervised_trainer INFO     Progress: 88%, Train Perplexity: 1.0000
2018-02-22 08:58:29,891 seq2

2018-02-22 08:58:31,158 seq2seq.trainer.supervised_trainer INFO     Progress: 92%, Train Perplexity: 1.0015
2018-02-22 08:58:31,171 seq2seq.trainer.supervised_trainer INFO     Progress: 92%, Train Perplexity: 1.0000
2018-02-22 08:58:31,181 seq2seq.trainer.supervised_trainer INFO     Progress: 92%, Train Perplexity: 1.0001
2018-02-22 08:58:31,193 seq2seq.trainer.supervised_trainer INFO     Progress: 92%, Train Perplexity: 1.0000
2018-02-22 08:58:31,211 seq2seq.trainer.supervised_trainer INFO     Progress: 92%, Train Perplexity: 1.0000
2018-02-22 08:58:31,233 seq2seq.trainer.supervised_trainer INFO     Progress: 92%, Train Perplexity: 1.0000
2018-02-22 08:58:31,243 seq2seq.trainer.supervised_trainer INFO     Progress: 92%, Train Perplexity: 1.0000
2018-02-22 08:58:31,268 seq2seq.trainer.supervised_trainer INFO     Progress: 92%, Train Perplexity: 1.0000
2018-02-22 08:58:31,279 seq2seq.trainer.supervised_trainer INFO     Progress: 92%, Train Perplexity: 1.0000
2018-02-22 08:58:31,294 seq2

2018-02-22 08:58:32,552 seq2seq.trainer.supervised_trainer INFO     Progress: 96%, Train Perplexity: 1.0000
2018-02-22 08:58:32,588 seq2seq.trainer.supervised_trainer INFO     Progress: 96%, Train Perplexity: 1.0000
2018-02-22 08:58:32,605 seq2seq.trainer.supervised_trainer INFO     Progress: 96%, Train Perplexity: 1.0000
2018-02-22 08:58:32,623 seq2seq.trainer.supervised_trainer INFO     Progress: 96%, Train Perplexity: 1.0000
2018-02-22 08:58:32,640 seq2seq.trainer.supervised_trainer INFO     Progress: 96%, Train Perplexity: 1.0000
2018-02-22 08:58:32,665 seq2seq.trainer.supervised_trainer INFO     Progress: 96%, Train Perplexity: 1.0000
2018-02-22 08:58:32,683 seq2seq.trainer.supervised_trainer INFO     Progress: 96%, Train Perplexity: 1.0000
2018-02-22 08:58:32,694 seq2seq.trainer.supervised_trainer INFO     Progress: 96%, Train Perplexity: 1.0000
2018-02-22 08:58:32,711 seq2seq.trainer.supervised_trainer INFO     Progress: 96%, Train Perplexity: 1.0000
2018-02-22 08:58:32,733 seq2

In [18]:
predictor = Predictor(seq2seq, input_vocab, output_vocab)

In [ ]:
while True:
    seq_str = raw_input("Type in a source sequence:")
    seq = seq_str.strip().split()
    print(predictor.predict(seq))

In [1]:
import os

In [2]:
data_dir="~/ijcnlp_dailydialog"
os.path.join(data_dir, 'daily_dialog')

'~/ijcnlp_dailydialog/daily_dialog'

In [30]:
final_output=[]
with open("/home/ubuntu/ijcnlp_dailydialog/train/dialogues_train.txt") as f:
    for line in f:
        conv = line.strip().lower().split('__eou__')
        output = [conv[i].strip()+'\t'+conv[i+1].strip() for i in range(len(conv)-2)]
        final_output += output
with open('small.txt','w+') as of:
    for item in final_output:
        of.write("%s\n" % item)

In [31]:
os.path.join('root','name','data.txt')

'root/name/data.txt'

In [32]:
words=set()
for sentence in final_output:
    words.update(sentence.replace('\t',' ').split(' '))

In [33]:
len(list(words))

22075

In [34]:
words=list(words)

In [35]:
with open('small.txt', 'w+') as fout:
    fout.write("\n".join(words))

In [1]:
import string

In [2]:
s='Hi! HOw are, you'

In [4]:
s.translate(string.punctuation)

'Hi! HOw are, you'

In [5]:
from string import punctuation
''.join(c for c in s if c not in punctuation)

'Hi HOw are you'